<a href="https://colab.research.google.com/github/gdabrow/Investments/blob/main/stockAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Information on companies listed on stock exchanges obtained from:**
* https://seekingalpha.com/
* https://www.zacks.com/
* https://finance.yahoo.com/

In [ ]:
# installation libraries
! pip install yfinance --upgrade --no-cache-dir &> /dev/null
! pip install beautifulsoup4 &> /dev/null
! pip install cloudscraper &> /dev/nul

In [ ]:
# import libraries
import yfinance as yf
import pandas as pd
from bs4 import BeautifulSoup
import cloudscraper
import urllib.parse
import plotly.graph_objects as go
import plotly.express as px
import random

scraper = cloudscraper.create_scraper(delay=10)

# disk connection
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**List with the data we are interested in**

In [ ]:
columns = ['Symbol','Price','recommendationKey', 'recommendationMean', 'targetLowPrice [%]',
           'targetMeanPrice [%]', 'targetMedianPrice [%]', 'targetHighPrice [%]', 'numberOfAnalystOpinions',
           'P/E', 'EPS next Y', 'EPS next 5Y', 'Profit Margin',
           'ROA', 'ROE', 'ROI', 'currentRatio', 'quickRatio', 'Insider Trans', 'Inst Trans']

**Selection tickers for analysis**

In [ ]:
# data from a file on disk
input='seekingalpha'
seekingalpha = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/'+input+'.xlsx')
Tickers = list(seekingalpha.Symbol)

**Downloading data from finviz and finance.yahoo**

In [ ]:
asset = pd.DataFrame(columns=columns)

for tic in Tickers:

  url = 'https://finviz.com/quote.ashx?t=' + tic
  page = scraper.get(url)
  page_html = BeautifulSoup(page.content, 'html.parser')
  table = page_html.find_all('tr', class_='table-dark-row')

  if table == []:
    continue

  names = []
  value = []

  # Downloading data from finviz (website source)
  for row in table:
    #for name in row.find_all('td', class_='snapshot-td2-cp'):
    #  names.append(name.text)
    i=1
    for name in row.find_all('td', class_='snapshot-td2'):
      if name.text == '-':
        v = 0
      else:
        v = name.text

      if i%2 == 1:
        names.append(name.text)
      else:
        value.append(v)
      i+=1

  # Data connection in dictionary
  finviz = dict(zip(names, value))
  finviz['Symbol']=tic

  # Downloading data from finance.yahoo API
  for _ in range(3):
    try:
      #data = yf.Ticker(tic).info
      data = yf.Ticker(tic).info
      break
    except:
      continue

  # Data connection finance.yahoo + finviz
  data.update(finviz)

# Conversion to percentages
  for_percentage = {'targetLowPrice [%]':'targetLowPrice', 'targetMeanPrice [%]':'targetMeanPrice',
                    'targetMedianPrice [%]': 'targetMedianPrice', 'targetHighPrice [%]':'targetHighPrice'}

  for col in for_percentage:
    try:
      data[col] = round(100 * (data[for_percentage[col]])/float(finviz['Price'])-100)
    except:
      data[col] = '-'

  row={}
  for col in columns:

    try:
      row[col] = data[col]
    except:
      row[col] = '-'

  asset = asset.append(row, ignore_index=True)


# Columns that need to be reformatted to number
colum = ['EPS next Y', 'EPS next 5Y', 'Profit Margin', 'ROA', 'ROE', 'ROI', 'Insider Trans', 'Inst Trans']

for col in colum:
  asset[col] = asset[col].str.replace("%", "").astype(float)

asset.set_index('Symbol', inplace=True)

<ipython-input-5-7fe22f88510c>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  asset = asset.append(row, ignore_index=True)
<ipython-input-5-7fe22f88510c>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  asset = asset.append(row, ignore_index=True)
<ipython-input-5-7fe22f88510c>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  asset = asset.append(row, ignore_index=True)
<ipython-input-5-7fe22f88510c>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  asset = asset.append(row, ignore_index=True)
<ipython-input-5-7fe22f88510c>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version.

**Downloading data from Zacks**

In [ ]:
# In case Zacks realizes the page is scrapped you can get help from scrapeops.io
import time
API_KEY = '5680dc92-6ddb-4d18-b643-d79951641a14'

def get_scrapeops_url(url):
  payload = {'api_key': API_KEY, 'url': url, 'bypass': 'cloudflare'}
  proxy_url = 'https://proxy.scrapeops.io/v1/?' + urllib.parse.urlencode(payload)
  return proxy_url

#Fake user agent for scraper
user_agents_list = ['Mozilla/5.0 (X11; Linux i686) AppleWebKit/5311 (KHTML, like Gecko) Chrome/40.0.844.0 Mobile Safari/5311']

zacks = []

for tic in Tickers:
  #url = 'https://www.zacks.com/stock/quote/' + tic
  url = 'https://www.zacks.com/stock/quote/'+tic+'?q='+tic
  try:
    #page = scraper.get(url, headers=HEADERS)
    page = scraper.get(url, headers={'User-Agent': random.choice(user_agents_list)})
    time.sleep(2)
    #page = scraper.get(get_scrapeops_url(url))
  except:
    zacks.append(0)
    time.sleep(2)
    continue

  page_html = BeautifulSoup(page.content, 'html.parser')
  table = page_html.find_all('p', class_="rank_view")
  try:
    zac = list(table[0].get_text().replace(" ", ""))[1]
  except IndexError:
    zacks.append(0)
    continue

  zacks.append(zac)

**Table formatting for future connection (if you use seekingalpha)**

In [ ]:
seekingalpha['zacks'] = zacks
seekingalpha.set_index('Symbol', inplace=True)
asset.rename_axis("Symbol", inplace=True)

#Table joins
asset = pd.merge(left=seekingalpha, right=asset, on='Symbol')

**Preparation of a table for scoring purposes**

In [ ]:
scoring = pd.DataFrame()

# Changing the ranking to the "more the better" and rescaling it to values between 0 and 1
# Check if the column is an object, if so, change "nan" to 100 and change to float

asset.recommendationMean.fillna(100, inplace=True)

if asset.recommendationMean.dtypes == 'O':
  scoring['recommendationMean'] = asset['recommendationMean'].str.replace(",", ".")
  scoring['recommendationMean'] = scoring['recommendationMean'].str.replace("-", "100")
  scoring['recommendationMean'] = scoring.recommendationMean.astype('float')
  scoring['recommendationMean'] = (1/scoring.recommendationMean)
else:
  scoring['recommendationMean'] = (1/asset.recommendationMean)

# Convert str to float and rescaling to values between 0 and 1

asset['Quant Ratings'].fillna(0, inplace=True)

if asset['Quant Ratings'].dtypes == 'O':
  scoring['Quant Ratings'] = asset['Quant Ratings'].str.replace(",", ".")
  scoring['Quant Ratings'] = scoring['Quant Ratings'].str.replace("-", "0").astype('float')
  scoring['Quant Ratings'] = scoring['Quant Ratings']/5
else:
  scoring['Quant Ratings'] = asset['Quant Ratings']/5


asset['SA Analyst Ratings'].fillna(0, inplace=True)

if asset['SA Analyst Ratings'].dtypes == 'O':
  scoring['SA Analyst Ratings'] = asset['SA Analyst Ratings'].str.replace(",", ".")
  scoring['SA Analyst Ratings'] = scoring['SA Analyst Ratings'].str.replace("-", "0")
  scoring['SA Analyst Ratings'] = scoring['SA Analyst Ratings'].astype('float')/5
else:
  scoring['SA Analyst Ratings'] = asset['SA Analyst Ratings']/5


asset['Wall Street Ratings'].fillna(0, inplace=True)

if asset['Wall Street Ratings'].dtypes == 'O':
  scoring['Wall Street Ratings'] = asset['Wall Street Ratings'].str.replace(",", ".")
  scoring['Wall Street Ratings'] = scoring['Wall Street Ratings'].str.replace("-", "0")
  scoring['Wall Street Ratings'] = scoring['Wall Street Ratings'].astype('float')/5
else:
  scoring['Wall Street Ratings'] = asset['Wall Street Ratings']/5

In [ ]:
def conversion(x, thresholds):

  """
    Converts the value of x to a value between (0,1) based on a thresholds array.

    Args:
      x (float) - value to converts
      thresholds - (length 3 array of int) - thresholds for conversion

    Returns:
      value between (0,1)
  """

  if pd.isna(x):
    return 0
  elif x <= 0:
    return 0
  elif 0 < x <= thresholds[0]:
    return .25
  elif thresholds[0]<x<=thresholds[1]:
    return .5
  elif thresholds[1]<x<=thresholds[2]:
    return .75
  else:
    return 1

In [ ]:
# Dictionary with thresholds to convert the corresponding columns
dictForConv = {'targetLowPrice [%]':[5,10,20],
                 'targetMeanPrice [%]':[20,40,60],
                 'targetMedianPrice [%]':[20,40,60],
                 'targetHighPrice [%]':[40,60,80],
                 'numberOfAnalystOpinions':[3,10,15],
                 'EPS next Y':[5,10,15],
                 'EPS next 5Y':[5,15,25],
                 'Profit Margin':[10,20,30],
                 'ROA':[10,20,30],
                 'ROE':[15,30,45],
                 'ROI':[15,30,40],
                 'currentRatio':[1,3,5],
                 'quickRatio':[1,3,5],
                 'Insider Trans':[5,10,20],
                 'Inst Trans':[5,10,20]}

In [ ]:
asset

,Quant Ratings,SA Analyst Ratings,Wall Street Ratings,Valuation Grade,Growth Grade,Profitability Grade,Momentum Grade,EPS Revision Grade,zacks,Price,...,EPS next Y,EPS next 5Y,Profit Margin,ROA,ROE,ROI,currentRatio,quickRatio,Insider Trans,Inst Trans
Symbol,,,,,,,,,,,,,,,,,,,,,
TAST,4.00,5.00,4.94,D,B-,C-,A+,A+,1,7.46,...,2.63,NaN,0.21,0.57,5.30,0.28,0.657,0.493,-0.14,-6.53
AER,4.60,4.70,4.62,A-,C,A+,B+,B+,2,66.97,...,2.89,5.40,34.46,3.57,15.85,3.95,1.989,1.485,NaN,-0.78
BEP,3.71,4.23,2.85,B,B,A-,D+,D+,3,24.58,...,-68.79,13.20,-2.88,-0.16,-2.09,-0.51,0.762,0.584,0.00,NaN
FTAI,4.00,4.70,4.47,D+,A,A+,A+,C,3,41.90,...,43.35,NaN,10.63,6.20,253.65,5.14,2.741,1.011,0.00,-1.59
NXPI,4.00,3.80,4.35,C-,C,A,B,B,3,199.66,...,4.72,7.84,21.43,12.02,36.19,15.12,1.921,1.257,-10.49,-2.38
PEP,3.76,3.65,4.75,D,C+,A+,C+,A-,3,168.32,...,7.98,8.64,9.05,8.53,43.88,15.17,0.877,0.676,-1.06,-0.02
RIO,3.85,4.80,3.24,A-,D-,A+,B+,C+,3,69.03,...,0.93,-3.30,16.39,8.82,17.23,13.49,1.989,1.374,NaN,4.93
OBDC,3.87,4.33,4.69,A,A-,A,B+,B+,1,14.73,...,-1.19,NaN,54.18,5.93,13.44,6.10,1.439,1.183,0.13,-4.44
CF,4.75,3.50,3.07,B-,C-,A+,B-,B-,3,75.90,...,-18.87,NaN,27.53,15.74,41.53,23.80,4.525,3.964,-5.48,-2.53


In [ ]:
# Column conversion
for col in dictForConv:

  asset[col].fillna(0, inplace=True)

  if asset[col].dtypes == 'O':
    asset[col] = asset[col].astype('string')
    scoring[col] = asset[col].str.replace(",", ".")
    scoring[col] = scoring[col].str.replace("-", "0").astype('float')
    scoring[col] = scoring[col].apply(lambda x: conversion(x,dictForConv[col]))
  else:
    scoring[col] = asset[col].apply(lambda x: conversion(x,dictForConv[col]))

In [ ]:
# Dictionaries to convert the remaining columns
seekingalphaMap = {'A+':1, 'A': .9, 'A-': .8, 'B+': .7, 'B': .6, 'B-': .5, 'C+': .4, 'C-': .3, 'D+': .2, 'D': .1, 'D-': 0, 'F': 0, '-':0}
colToConvert = ['Valuation Grade',	'Growth Grade',	'Profitability Grade', 'Momentum Grade']

# conversion
for col in colToConvert:
  scoring[col] = asset[col].map(seekingalphaMap)

# Dictionery to convert 'zacks' column
#zacksMap = {'1':1, '2':.75, '3':.5, '4':.25, '5':0}
zacksMap = {1:1, 2:.75, 3:.5, 4:.25, 5:0, 0:0}

# conversion
scoring['zacks'] = asset['zacks'].map(zacksMap)

In [ ]:
weightDict = {'recommendationMean':1.2,
           'Quant':1.3,
           'SA Authors':1,
           'Wall St.':1.3,
           'targetLowPrice [%]':1,
           'targetMeanPrice [%]':1.3,
           'targetMedianPrice [%]':1.3,
           'targetHighPrice [%]':1,
           'numberOfAnalystOpinions':1,
           'EPS next Y':1.3,
           'EPS next 5Y':1.2,
           'Profit Margin':1,
           'ROA':1,
           'ROE':1,
           'ROI':1,
           'currentRatio':1,
           'quickRatio':1,
           'Insider Trans':1,
           'Inst Trans':1,
           'Valuation':1,
           'Growth':1,
           'Profitability':1,
           'Momentum':1,
           'zacks':1.2}

weight = list(weightDict.values())


**Weighing and adding scoring**

In [ ]:
to_score = ['recommendationMean','Quant Ratings','SA Analyst Ratings','Wall Street Ratings','targetLowPrice [%]',
            'targetMeanPrice [%]', 'targetMedianPrice [%]', 'targetHighPrice [%]','zacks']

scoring = scoring * weight
scoring = scoring[to_score]
scoring = scoring.sum(axis=1)
asset['scoring']= scoring
asset.sort_values(by='scoring', ascending=False, inplace=True)

In [ ]:
fig = px.bar(asset, x=asset.index, y='scoring')
fig.show()

**Saving data to disk**

In [ ]:
output='top'

In [ ]:
asset.to_excel('/content/drive/MyDrive/Colab Notebooks/output_'+output+'.xlsx')

**Data visualization**

In [ ]:
def chart(symbol):

  tic = yf.Ticker(symbol)
  data = tic.history(period = "ytd")
  data['targetLowPrice'] = asset.loc[symbol, 'targetLowPrice']
  data['targetMeanPrice'] = asset.loc[symbol, 'targetMeanPrice']
  data['targetHighPrice'] = asset.loc[symbol, 'targetHighPrice']

  fig = go.Figure()

  fig.add_trace(go.Scatter(x=data.index, y=data.Close,
                             mode='lines',
                             name='Close'))

  fig.add_trace(go.Scatter(x=data.index, y=data['targetLowPrice'],
                             mode='lines',
                             name=asset.loc[symbol, 'targetLowPrice [%]']))

  fig.add_trace(go.Scatter(x=data.index, y=data['targetMeanPrice'],
                           mode='lines',
                           name=asset.loc[symbol, 'targetMeanPrice [%]']))

  fig.add_trace(go.Scatter(x=data.index, y=data['targetHighPrice'],
                           mode='lines',
                           name=asset.loc[symbol, 'targetHighPrice [%]']))
  fig.show()

In [ ]:
# chart titles
title = ['Recommendation mean', 'Target low price [%]',
           'Target mean price [%]', 'Target median price [%]', 'Target high price [%]',
           'P/E', 'EPS next Y', 'EPS next 5Y', 'Profit Margin',
           'ROA', 'ROE', 'ROI', 'Current ratio', 'Quick ratio']

col_tite = dict(zip(columns[3:], title))

for statistics in col_tite:
  fig = px.bar(asset, x=asset.index, y=statistics, title=col_tite[statistics])
  fig.show()

In [ ]:
!pip -qqq install tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 78.5 MB/s eta 0:00:00


In [ ]:
from tabula import read_pdf

In [ ]:
url = 'https://www.mdm.pl/bm/g/uOdhuGhZhU6oJPIdXBx5rA'

In [ ]:
data = read_pdf(url, pages=[1,2])

In [ ]:
len(data)

2

In [ ]:
data = data[0].iloc[8:]

In [ ]:
columns = ['GIEŁDA (WALUTA)',
           'NAZWA RYNKU W SERWISIE INWESTORA',
           'NAZWA INSTRUMENTU',
           'NAZWA INSTRUMENTU W SERWISIE INWESTORA',
           'TICKER',
           'KOD ISIN INSTRUMENTU',
           'TYP INSTRUMENTU',
           'RATING MSCI',
           'ESG',
           'CZY UWZLĘDNIA CZYNNIKI ZRÓWNOWAŻONEGO ROZWOJU (ESG)*?']

In [ ]:
data.columns = columns

In [ ]:
data[['TICKER BLOOMBERG', 'TICKER GOOGLE']] = data['TICKER'].str.split(' ', expand=True)

In [ ]:
data.drop('TICKER', axis=1, inplace=True)

In [ ]:
data

,GIEŁDA (WALUTA),NAZWA RYNKU W SERWISIE INWESTORA,NAZWA INSTRUMENTU,NAZWA INSTRUMENTU W SERWISIE INWESTORA,KOD ISIN INSTRUMENTU,TYP INSTRUMENTU,RATING MSCI,ESG,CZY UWZLĘDNIA CZYNNIKI ZRÓWNOWAŻONEGO ROZWOJU (ESG)*?,TICKER BLOOMBERG,TICKER GOOGLE
8,DEUTSCHE BÖRSE (EUR),DEU-XETRA,TELEFONICA DEUTSCHLAND HOLDING AG,TELEFON DEUT,DE000A1J5RX9,AKCJA,NaN,AA,TAK,O2D:GR,O2D
9,DEUTSCHE BÖRSE (EUR),DEU-XETRA,TELEFONICA SA,TELEFONICA,ES0178430E18,5)AKCJA,NaN,A,TAK,TNE5:GR,TNE5
10,DEUTSCHE BÖRSE (EUR),DEU-XETRA,TELIA CO AB,TELIA CO AB,SE0000667925,AKCJA,NaN,AAA,TAK,TLS:GR,TLS
11,DEUTSCHE BÖRSE (EUR),DEU-XETRA,THYSSENKRUPP AG,THYSSENKRUPP,DE0007500001,AKCJA,NaN,AA,TAK,TKA:GR,TKA
12,DEUTSCHE BÖRSE (EUR),DEU-XETRA,TLG IMMOBILIEN A,TLG IMMOBIL,DE000A12B8Z4,AKCJA,NaN,BBB,NIE,TLG:GR,TLG
13,1)DEUTSCHE BÖRSE (EUR),DEU-XETRA,TOTAL SA,TOTAL SA,FR0000120271,AKCJA,NaN,AA,TAK,TOTB:GR,TOTB
14,DEUTSCHE BÖRSE (EUR),DEU-XETRA,TUI AG,TUI AG,DE000TUAG505,AKCJA,NaN,BB,NIE,TUI1:GR,TUI1
15,1)DEUTSCHE BÖRSE (EUR),DEU-XETRA,UBISOFT ENTERTAINMENT SA,UBISOFT,FR0000054470,AKCJA,NaN,AA,TAK,UEN:GR,UEN
16,2)DEUTSCHE BÖRSE (EUR),DEU-XETRA,UNICREDIT SPA,UNICREDIT,IT0005239360,AKCJA,NaN,AA,TAK,CRIN:GR,CRIN
17,DEUTSCHE BÖRSE (EUR),DEU-XETRA,UNIPER SE,UNIPER SE,DE000UNSE018,AKCJA,NaN,BBB,NIE,UN01:GR,UN01
